# Final graded assignment: 
# _Input-output properties of a layer 4 pyramidal cell_

Copyright © 2005-2021 Blue Brain Project/EPFL. This notebook is under a [LGPLv3](https://raw.githubusercontent.com/BlueBrain/MOOC-neurons-and-synapses-2017/master/LICENSE.txt) license.

Using techniques from the previous week 6 exercise on the Layer 5 pyramidal cell from the NMC portal, in this assignment you will simulate a layer 4 pyramidal cell from the NMC portal under several input configurations, and assess its output.

Let's begin with installing and importing the required Python packages

In [ ]:
from tqdm import tqdm

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
from tqdm import tqdm
import zipfile # Extract zip files
import urllib # Download files from the web
import neurom # Analyse / view morphologies
import neurom.viewer

## Downloading the model

As in the Week 6 Tutorial, let's download a Neocortical Layer 4 Pyramidal cell model from the NMC Portal:

In [ ]:
urllib.request.urlretrieve('https://bbp.epfl.ch/nmc-portal/assets/documents/static/downloads-zip/L4_PC_cADpyr230_2.zip','L4_PC_cADpyr230_2.zip');

Then we extract this zip file on disk:

In [ ]:
import os, zipfile
with zipfile.ZipFile('L4_PC_cADpyr230_2.zip', 'r') as zip_file:
    zip_file.extractall('.')

Now we change directory to the model dir. (**Be careful**, once this line is executed don't execute the lines above because it will create nested subdirectories)

In [ ]:
os.chdir('L4_PC_cADpyr230_2')

The model contains MOD files describing the NEURON simulator mechanisms used in the model. Compile these files using the 'nrnivmodl' command:

In [ ]:
!nrnivmodl mechanisms

## Visualizing the morphology

The morphology of the cell is located in the 'morphology' subdirectory. 
We can visualize this morphology using NeuroM:

In [ ]:
neurom.viewer.draw(neurom.load_neuron('morphology/dend-C310897B-P3_axon-C220498B-P3_cor_-_Clone_1.asc'));

## Initializing the model

To load the functionality of the electrical model we load the 'init.hoc' HOC file.

In [ ]:
%%capture
import neuron as nrn # NEURON simulator
nrn.h.load_file("./init.hoc")

The init.hoc has created a function 'create_cell' to instantiate the neuron in the NEURON simulator. As argument it takes a value to enable/disable the synapses of the cell. (1=True, 0=False)

In [ ]:
%%capture
nrn.h.create_cell(1) # argument 1 stands for 'load synapses'

After this function has been called the global name space of the NEURON simulator contains a variable 'cell'. 
Let's create a python variable to point to this object and for the soma:

In [ ]:
cell = nrn.h.cell
soma = cell.soma[0]

## Injecting a step current

Now that we have a pointer to the soma of the neuron, we can inject a current into it. As we've learned in another tutorial, we can create an IClamp to inject the current:

In [ ]:
stimulus = nrn.h.IClamp(0.5, sec=soma)

In [ ]:
stimulus.dur = 100000 # ms
stimulus.delay = 100  # ms     

As in the tutorial, let's retrieve the holding and step currents from a file in the model package:

In [ ]:
with open('current_amps.dat') as current_file:
    current_content = current_file.read()

print('File content: ', current_content)
holding_current, step1_current, step2_current, step3_current = [float(x) for x in current_content.split()]

print('')
print('Holding current: %f nA' % holding_current)
print('Step 1: %f nA' % step1_current)
print('Step 2: %f nA' % step2_current)
print('Step 3: %f nA' % step3_current)

The 'holding current' is the current the experimentalists inject during the entire recording to hold the neuron at a predefined voltage before the actual stimulus is applied.

The init.hoc also has provided us with a function to enable the recording of the voltage:

In [ ]:
nrn.h.create_recording()    #activate recording of activity 

In what follows, we will run some fairly long simulations.
Let's define a function to run simulations with a progress bar:

In [ ]:
def run_progressbar(tstop=600):
    # make windows approimately 50ms long
    num_windows = int(float(tstop)/50)
    if num_windows<10:
        num_windows=10
    windows = numpy.linspace(0,tstop,num_windows)
    for nrn.h.tstop in tqdm(windows[1:]):
        if nrn.h.tstop==windows[1]:
            # first run
            nrn.h.run()
        else:
            nrn.h.continuerun(nrn.h.tstop)

... and define a convenience function to simulate and plot

In [ ]:
time = nrn.h.time
voltage = nrn.h.voltage

def plot_tv(time_array, voltage_array, show=True, label=None, constants=[]):
    plt.plot(time_array, voltage_array, label=label)
    for constant in constants:
        plt.plot(time_array, constant*numpy.ones(len(time_array)))
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane voltage (mV)')
    if show:
        plt.show()
        
def run_plot(tstop=600):
    nrn.h.dt = 0.05

    run_progressbar(tstop)
    nrn.h.save_recording()

    plot_tv(time, voltage)

First let's stimulate the cell with the first step current protocol

In [ ]:
stimulus.amp = holding_current + step1_current
run_plot(10000)

## Exercise Question 1
Using a threshold of 0mV, detect the spike times in the voltage train.  Throw away spike times less that 1s, as they represent a transient phase where the system relaxes from its initial conditions.  Compute the time intervals between subsequent spikes, known as the inter-spike intervals (ISI).  That is, if $t_i$ are the spike times, then $ISI_i=t_{i+1}-t_i$.  This offset subtraction can be compactly represented using numpy indexing of the form `y = x[1:] - x[:-1]`.  Try it out!  

Plot the histogram of ISIs for your spike dataset. How does it look? A Poisson process has an exponential ISI distribution, and a coefficient of variation (CV; see week 4) of 1.  Spiking more "regular" than Poisson will have a CV less than 1.  Spiking more "irregular" than Poisson will have a CV greater than 1.  Compute the CV for your simulation, and add it to an **ans** dictionary as a string or float with key "Q1".

In [ ]:
ans = {}
ans["Q1"] = "my CV"

## Activating synapses
Now let's define an _in vivo_-like stimulus, where input neurons are firing like Poisson processes with presynaptic morphology type (mtype) specific rates.  

Following the tutorial, load and build the mtype id map:

In [ ]:
with open('synapses/mtype_map.tsv') as mtype_map_file:
    mtype_map_content = mtype_map_file.read()
    
mtype_map = {}
for line in mtype_map_content.split('\n')[:-1]:
    n, mtype = line.split()
    mtype_map[mtype] = int(n)
    
print(mtype_map)

Now, following the tutorial, we will enable all the synapses that are projected onto this cell from all mtypes.
To this end we will define, as before, the init_synapses function:

In [ ]:
def init_synapses(enabled_mtypes=[]):
    """Enable all the synapses that are projected onto this cell from mtype listed in enabled_mtypes."""
    enabled_mtype_ints = [mtype_map[mtype] for mtype in enabled_mtypes]
    
    for i in range(0, int(cell.synapses.n_of_mtypes)): # Loop over all the m-type
        if i in enabled_mtype_ints: # Enable synapses
            #  The [were_]active_pre_mtypes is a NEURON vector 
            # (it uses the .x syntax to access the elements)
            # When the value in the vector is 1 all the presynaptic neurons
            # of a particular m-types are active (and inactive when it is 0)
            cell.synapses.were_active_pre_mtypes.x[i]= 0
            cell.synapses.active_pre_mtypes.x[i] = 1        
        else: # Disable synapses
            cell.synapses.were_active_pre_mtypes.x[i]= 1
            cell.synapses.active_pre_mtypes.x[i] = 0
            
    cell.synapses.update_synapses(nrn.h.synapse_plot); # Update the synapses


In [ ]:
# Enable incoming synapses from all mtypes
init_synapses(enabled_mtypes=mtype_map.keys())

In [ ]:
# The in vivo fluctation driven regime: 
# Depolarize at approx. 100%
# firing times are determined by synaptic drive "fluctuations"
stimulus.amp = holding_current + (step1_current/1.2) # inject the 100% threshold
stimulus.dur = 100000 # keep it on

Each of the synapses is driven by a Poisson process with a firing rate that can be configured for each presynaptic mtype.  Let's configure mtype specific firing rates as described in [Gentet et al., 2010](http://www.sciencedirect.com/science/article/pii/S0896627310000115) for awake conditions, i.e. about 1 Hz for excitatory cells, and 5.0 Hz for inhibitory cells.

In [ ]:
exc_cells = ['L23_PC', 'L4_PC', 'L4_SS', 'L4_SP', 
             'L5_TTPC1', 'L5_TTPC2', 'L5_STPC', 'L5_UTPC',
             'L6_TPC_L1', 'L6_TPC_L4', 'L6_BPC', 'L6_IPC', 'L6_UTPC']
for mtype in mtype_map:
    if mtype in exc_cells:
        freq = 1.0 # [Hz]
    else:
        freq = 5.0 # [Hz]
    cell.synapses.pre_mtype_freqs.x[mtype_map[mtype]]=freq
    
cell.synapses.update_synapses(nrn.h.synapse_plot);

Now run 100s of simulation, to collect ample statistics.  Note: this will take about 10 minutes to complete.

In [ ]:
run_plot(100000)

## Exercise Question 2
As in **Question 1**, plot the histogram of ISIs of your spike dataset for this _in vivo_-like stimulation.  How does it look? Compute the CV.  A Poisson process has an exponential ISI distribution, and a coefficient of variation (CV; see week 4) of 1.  Add your answer to the ans dictionary as a string or float with key "Q2". 

In [ ]:
ans["Q2"]=cv

## Statistical models for neuronal firing

The gamma process, with ISI statistics governed by a gamma probability distribution function (PDF) or gamma distribution, is often used to describe the ISI statistics of neuronal firing.  The gamma distribution has two parameters, shape and scale, which parameterize the distribution as described [here](https://en.wikipedia.org/wiki/Gamma_distribution).  The shape and scale parameters uniquely determine the mean and standard deviation of the distribution and vice versa: $\mu=k\theta$, $\sigma=\sqrt{k\theta^2}$, where $\mu$ and $\sigma$ are the mean and standard deviation, and $k$ and $\theta$ are the shape and scale parameters, respectively.  

The various stats of the [gamma process](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html) are available in scipy:


In [ ]:
from scipy.stats import gamma

The gamma process provided by scipy has an additional parameter, loc (which stands for location), which implements a translation of the PDF along the x-axis.  This can be used to describe  neurons which exhibit a refractory period, a time interval following a spike for which spiking is forbidden. 

Assuming your inter-spike interval data are stored in the variable `isi`,
the following command can be used to fit a gamma distribution to your ISI data:

`shape, loc, scale = gamma.fit(isi)`

For a pure gamma distribution, one must force a zero refractory period, i.e. loc=0, for the fit as follows:


In [ ]:
shape, loc, scale = gamma.fit(isi, floc=0)
print(shape)
print(scale)

In [ ]:
# For the gamma distribution, shape and scale can also be computed from mean and variance 
shape2 = numpy.mean(isi)**2/numpy.var(isi)
print(shape2)
scale2 = numpy.var(isi)/numpy.mean(isi)
print(scale2)

Now, lets compare our fit to a histogram of our ISI data, to visually inspect how well the gamma process describes the firing of our model neuron.

In [ ]:
# The 'denstity' kwarg results in a pdf, i.e. its normalized to 1
# so it can be directly compared to gamma.pdf 
h1, bins = numpy.histogram(isi, bins=50, density=True);
# append a zero bin at the end to make same dimensions as bins (for plotting)
h1 = numpy.hstack((h1, 0))

In [ ]:
plt.plot(bins, h1, ls="--")
x = numpy.arange(0,1000,0.5)
plt.plot(x, gamma.pdf(x, a=shape, loc=loc, scale=scale))

## Exercise Question 3
Following the development above, fit a gamma process with refractory period (loc>=0) to your isi data.  From visual inspection, does the additional refractory period parameter make much difference in describing your isi data for short isi intervals?  Submit your resulting shape, loc, scale fits as follows: 

In [ ]:
ans["Q3"]=(shape, loc, scale)

## ISI Correlations

The gamma process is a special case of a renewal process, and as such, subsequent ISIs are statistically independent.  By contrast, neuronal ISIs are generally negatively correlated, as a result of [spike frequency adaptation](http://www.scholarpedia.org/article/Spike_frequency_adaptation) due to spike activated $K^+$ channels such as the [M-current](http://channelpedia.epfl.ch/ionchannels/26). Let's investigate this concept more closely.

To begin, let's generate some ISIs for our fitted gamma process, and plot a scatter plot of $ISI_{i+1}$ vs $ISI_i$.  To this end, the `gamma.rvs` can generate these "random variates" according to the parameters we fit previously.

In [ ]:
gamma_isi = gamma.rvs(size=isi.shape, a=shape, loc=loc, scale=scale)

In [ ]:
plt.plot(gamma_isi[:-1], gamma_isi[1:], '.')
plt.xlabel('$ISI_i$ [ms]', fontsize=16)
plt.ylabel('$ISI_{i+1}$ [ms]', fontsize=16)

We can assess if subsequent ISIs of the gamma process are correlated using a linear regression:

In [ ]:
from scipy.stats import linregress

In [ ]:
slope, intercept, r_value, p_value, std_err = linregress(gamma_isi[:-1], gamma_isi[1:])

In [ ]:
r_value, p_value 

As described [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.linregress.html), the r_value is the correlation coefficient, and the p_value is for the test of the null hypothesis (no correlation).  We find an r_value close to zero, and a high p_value indicating we cannot reject the null hypothesis that subsequent ISIs are uncorrelated.  Indeed they ISIs are uncorrelated by definition for the gamma process.

Let's plot the result of the linear regression:

In [ ]:
plt.plot(gamma_isi[:-1], gamma_isi[1:], '.')
plt.plot(bins, slope*bins+intercept, 'r-')

Let's now generate the scatter plot for our simulated neuronal ISI data

In [ ]:
plt.plot(isi[:-1], isi[1:], '.')
plt.xlabel('$ISI_i$ [ms]', fontsize=16)
plt.ylabel('$ISI_{i+1}$ [ms]', fontsize=16)

We get a clear picture that long ISIs ($>500$ ms) are followed by short ISIs ($<300$ ms), whereas short ISIs ($<300$ ms) are followed by the longest ISIs, sometimes $>700$ ms.  This suggests that subequent ISIs are negatively correlated.  We can again investigate this correlation more quantitatively using linear regression.

## Exercise Question 4
Undertake a similar analysis as above for your simulated neuronal ISIs.  Perform a linear regression on subsequent ISIs.  Submit your r_value and p_value for your neuronal ISI data as follows.  Consider the result: Does linear regression reveal a statistically significant negative correlation, as expected?  

In [ ]:
ans["Q4"]=(r_value, p_value)

## Submit you answers
Print your **ans** dictionary as a json string, and submit the string to the course platform by executing the submission code below.  You will also need a submission key from the course platform. 

In [ ]:
import json; print(json.dumps(ans))

In [ ]:
# Submit your answer.  The submission key can be retrieved from the edX platform
import single_cell_mooc_client as sc_mc
submission_widget = sc_mc.Submission()